# 102Flowers Image Classifier

This is the main notebook for the project. See the associated report (WIP) for more information.

WORK IN PROGRESS

### Imports

In [533]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import datasets, transforms, models

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy

### Hyperparameters

In [534]:
# TODO: Set hyperparameters.
training_batch_size = 20
validation_batch_size = 20
test_batch_size = 20
epochs = 20
learning_rate = 0.001
#momentum = 0.9
crop_size = 128

### Device

In [535]:
# Default to CPU
device = torch.device("cpu")

# Switch to GPU if available
if torch.cuda.is_available():
	print(f"Found {torch.cuda.device_count()} GPUs. Using cuda:0.")
	device = torch.device("cuda:0")
else:
	print("No GPUs found, using CPU.")

Found 1 GPUs. Using cuda:0.


### Load Dataset

In [536]:
training_data = datasets.Flowers102(
    root = "data",
    split = "train",
    transform=transforms.Compose([
        transforms.Resize(crop_size),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
    ]),
    download=True
)

validation_data = datasets.Flowers102(
    root = "data",
    split = "val",
    transform=transforms.Compose([
        transforms.Resize(crop_size),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
    ]),
    download=True
)

testing_data = datasets.Flowers102(
    root = "data",
    split = "test",
    transform=transforms.Compose([
        transforms.Resize(crop_size),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
    ]),
    download=True
)

### DataLoaders

In [537]:
train_dataloader = DataLoader(training_data, batch_size=training_batch_size, shuffle=True)
validation_dataloader = DataLoader(validation_data, batch_size=validation_batch_size, shuffle=True)
test_dataloader = DataLoader(testing_data, batch_size=test_batch_size, shuffle=True)

## Model

In [538]:
classifications = F.one_hot(torch.tensor([e for e in range(0,102)], device="cuda:0"), num_classes=102)
classifications, classifications.size()

(tensor([[1, 0, 0,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 0, 0, 0],
         [0, 0, 1,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 1, 0, 0],
         [0, 0, 0,  ..., 0, 1, 0],
         [0, 0, 0,  ..., 0, 0, 1]], device='cuda:0'),
 torch.Size([102, 102]))

In [539]:
class F102Classifier(nn.Module):
    
	def __init__(self):
		super(F102Classifier, self).__init__()
		
		self.pool = nn.MaxPool2d(8, 2)
		self.conv1 = nn.Conv2d(3, 6, 3) #3 inputs 6 hiddens
		self.BatchNorm = nn.BatchNorm2d(6) #Normalizes above
		self.conv2 = nn.Conv2d(6, 12, 3) # 12 hiddens
		self.conv3 = nn.Conv2d(12, 24, 3) # 24 hiddens
		self.fc1 = nn.Linear(15000, 102) #102 output neurons

	def forward(self, x):
		x = (F.relu(self.conv1(x)))
		#print("after conv1: ", x.size())
		x = self.BatchNorm(x)
		x = self.pool(F.relu(self.conv2(x)))
		#print("after conv2: ", x.size())
		x = self.pool(F.relu(self.conv3(x)))
		#print("after conv3: ", x.size())
		x = x.view(training_batch_size, -1)
		#print("after view: ", x.size())
		x = self.fc1(x)
		#print("after fc1: ", x.size())

		return x

net = F102Classifier()
if device == torch.device("cuda:0"):
	net.cuda()

### Loss Function & Optimiser

In [540]:
loss_function = nn.CrossEntropyLoss()
optimiser = optim.Adam(net.parameters(), lr=learning_rate)

## Train

In [541]:
def train(dataloader, model, loss_fn, optimizer):
	epoch = 1
	for batch, (i,j) in enumerate(dataloader):
		features, labels = i.to(device), j.to(device)
  
		# Compute the loss based off the predictions vs labels
		predictions = model(features)
		loss = loss_fn(predictions, labels)

		#Compute back propagation
		optimizer.zero_grad()
		loss.backward()
		optimizer.step()

		if (batch+1) % 51 == 0:
			print(f'Average Loss in Epoch: {loss.item()}')
print('Finished Training')

Finished Training


### Testing

In [542]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.cuda()
    model.eval()
    test_loss, correct = 0, 0
    for batch, (i,j) in enumerate(dataloader):
        features, labels = i.to(device), j.to(device)
        model.cuda()
        pred = model(features)
        test_loss += loss_fn(pred, labels).item()
        correct += (pred.argmax(1) == labels).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss : {test_loss:>8f} \n")

### Running Training and Testing

In [543]:
for t in range(epochs):
    print(f'Epoch {t+1}-------------')
    train(train_dataloader, net, loss_function, optimiser)

test(validation_dataloader, net, loss_function)

Epoch 1-------------
Average Loss in Epoch: 4.626456260681152
Epoch 2-------------
Average Loss in Epoch: 4.7379350662231445
Epoch 3-------------
Average Loss in Epoch: 3.9942638874053955
Epoch 4-------------
Average Loss in Epoch: 3.6112289428710938
Epoch 5-------------
Average Loss in Epoch: 2.247495412826538
Epoch 6-------------
Average Loss in Epoch: 2.269381284713745
Epoch 7-------------
Average Loss in Epoch: 0.7683330774307251
Epoch 8-------------
Average Loss in Epoch: 0.289736807346344
Epoch 9-------------
Average Loss in Epoch: 0.21078446507453918
Epoch 10-------------
Average Loss in Epoch: 0.09614106267690659
Epoch 11-------------
Average Loss in Epoch: 0.06913961470127106
Epoch 12-------------
Average Loss in Epoch: 0.06997961550951004
Epoch 13-------------
Average Loss in Epoch: 0.04501510411500931
Epoch 14-------------
Average Loss in Epoch: 0.0034210074227303267
Epoch 15-------------
Average Loss in Epoch: 0.0010489567648619413
Epoch 16-------------
Average Loss in Epoc

### Save Model

In [544]:
""" 30,0.001 = 15.2%
 100, 0.001 = 14.3%
 50, 0.001 = 16.4%
 30,0.01 = 1.0%  Herma-OF"""

"""For Theo	

        self.pool = nn.AvgPool2d(2, 2)
		self.conv1 = nn.Conv2d(3, 6, 3) 
		self.conv2 = nn.Conv2d(6, 12, 3)
		self.conv3 = nn.Conv2d(12, 24, 3)
		self.conv4 = nn.Conv2d(24, 48, 3)
		self.conv5 = nn.Conv2d(48, 96, 3)
		self.fc1 = nn.Linear(384, 1024)
		self.fc2 = nn.Linear(1024, 512)
		self.fc3 = nn.Linear(512, 102)	
30, 0.01 = 1.0%
30, 0.001 = 14.0%
100, 0.001 = 13.2%
30, 0.0001 = 7.2%

		self.pool = nn.AvgPool2d(2, 2)
		self.conv1 = nn.Conv2d(3, 12, 3)
		self.conv2 = nn.Conv2d(12, 48, 3)
		self.conv3 = nn.Conv2d(48, 96, 3)
		self.fc1 = nn.Linear(18816, 1024)
		self.fc2 = nn.Linear(1024, 512)
		self.fc3 = nn.Linear(512, 102)		
100, 0.001 = 9.5%
30, 0.001 = 17.8%
50, 0.001 = 1.0%

		self.pool = nn.AvgPool2d(2, 2)
		self.conv1 = nn.Conv2d(3, 12, 3)
		self.fc1 = nn.Linear(47628, 102)
30, 0.001 = 19.9%

		self.pool = nn.AvgPool2d(2, 2)
		self.conv1 = nn.Conv2d(3, 6, 3)
		self.fc1 = nn.Linear(23814, 102)
30, 0.001 = 17.9%
15, 0.001 = 16.0%
10, 0.001 = 19.0%
5, 0.001 = 14.4%

		self.pool = nn.MaxPool2d(2, 2)
		self.conv1 = nn.Conv2d(3, 6, 3)
		self.fc1 = nn.Linear(23814, 102)
30, 0.001 = 19.2%
10, 0.001 = 16.8%
50, 0.001 = 17.0

		self.pool = nn.MaxPool2d(2, 2)
		self.conv1 = nn.Conv2d(3, 6, 3)
		self.conv2 = nn.Conv2d(6, 12, 3)
		self.conv3 = nn.Conv2d(12, 24, 3)
		self.conv4 = nn.Conv2d(24, 48, 3)
		self.conv5 = nn.Conv2d(48, 96, 3)
		self.fc1 = nn.Linear(384, 102)
30, 0.001 = 18.6%

MORE
self.pool = nn.MaxPool2d(2, 2)
		self.conv1 = nn.Conv2d(3, 6, 3)
		self.conv2 = nn.Conv2d(6, 12, 3)
		self.conv3 = nn.Conv2d(12, 24, 3)
		self.fc1 = nn.Linear(21600, 102)
	def forward(self, x):
		x = (F.relu(self.conv1(x)))
		x = self.pool(F.relu(self.conv2(x)))
		x = self.pool(F.relu(self.conv3(x)))
		x = x.view(training_batch_size, -1)
		x = self.fc1(x)
        5 epochs = 10.5%
        10 epochs = 17.2%
        30 epochs = 16.1%
        15 epochs = 10.9
        
        Same as above but kernel size 4 and self.fc1 = nn.Linear(18816, 102)
        10 epochs = 9.2%
        15 epochs = 19.2%
        20 epochs = 21.5%
        25 epochs = 14.5%
        30 epochs = 18.4%
        
        Same as above burt kernal size 8 and self.fc1 = nn.Linear(15000, 102)
        20 epochs = 15.4 %
        30 epochs = 25.5%
        35 epochs = 16.1%
        
        Same as above but with batch normilization after conv1
        20 epochs = 12.9%
        25 epochs = 25.0%
        30 epochs = 19.6%
        40 epochs = 18.0%
 """
save_path = "./models/classifier.pth"
torch.save(net.state_dict(), save_path)